In [1]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [1]:
import json
from pycocotools.coco import COCO
import os
from PIL import Image

In [6]:
baseDir = 'coco\\train'
trainDir = os.path.join(baseDir, 'train2017')
dataType = 'train2017'
annFile = os.path.join(baseDir, 'annotations', f'instances_{dataType}.json')

In [7]:
coco = COCO(annFile)

loading annotations into memory...
Done (t=15.49s)
creating index...
index created!


In [44]:
import shutil
target_dir = 'data\\train'
desired_classes = ['person', 'dog', 'cat']
class_ids = coco.getCatIds(catNms=desired_classes)
for category in desired_classes:
    class_id = coco.getCatIds(catNms=[category])
    img_ids = coco.getImgIds(catIds=class_id)
    img_ids = list(set(img_ids))
    target_class_dir = os.path.join(target_dir, category)
    for image_id in img_ids:
        image_info = coco.loadImgs(image_id)[0]
        image_path = os.path.join(trainDir, image_info['file_name'])
        shutil.copy(image_path, target_class_dir)
    

In [68]:
testDir = 'coco\\test'
annFile = os.path.join(testDir, 'annotations', f'image_info_test2017.json')
testDir = 'coco\\test\\test2017'

In [69]:
coco = COCO(annFile)

loading annotations into memory...
Done (t=0.10s)
creating index...
index created!


In [71]:
desired_classes = ['person', 'dog', 'cat']
class_ids = coco.getCatIds(catNms=desired_classes)
img_ids = coco.getImgIds(catIds=1)
img_ids

[]

In [60]:
target_dir = 'data\\test'
desired_classes = ['person', 'dog', 'cat']
class_ids = coco.getCatIds(catNms=desired_classes)
for category in desired_classes:
    class_id = coco.getCatIds(catNms=[category])
    img_ids = coco.getImgIds(catIds=class_id)
    img_ids = list(set(img_ids))
    target_class_dir = os.path.join(target_dir, category)
    for image_id in img_ids:
        image_info = coco.loadImgs(image_id)[0]
        image_path = os.path.join(test, image_info['file_name'])
        shutil.copy(image_path, target_class_dir)

In [11]:
import numpy as np

In [12]:
image_size = (224, 224)

In [46]:
import tensorflow as tf

In [47]:
pretrained_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False)
pretrained_model.summary()

9406464/9406464 [==============================] - 1s 0us/step
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 Conv1 (Conv2D)                 (None, None, None,   864         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, None, None,   128         ['Conv1[0][0]']                  
                

In [48]:
pretrained_model.trainable = False

In [91]:
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam

x = pretrained_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation='softmax')(x)
model = Model(inputs=pretrained_model.input, outputs=predictions)

In [92]:
filepath="inceptionv3_best.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min')
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True, mode='min')
opt = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer = opt, 
              metrics=["accuracy"])


c:\Users\Tea_W\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [93]:
from keras.preprocessing.image import ImageDataGenerator

In [94]:
train_data_generator = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True, 
        width_shift_range = 0.2, 
        height_shift_range = 0.2,
        shear_range = 0.2, 
        zoom_range = 0.2, 
        validation_split=0.3)

test_data_generator = ImageDataGenerator(rescale=1./255)

train_data = train_data_generator.flow_from_directory('data/train/', target_size=(224, 224), shuffle=True, 
                                                       subset='training', class_mode='categorical')


Found 50831 images belonging to 3 classes.


In [95]:
validation_data = train_data_generator.flow_from_directory('data/train/', target_size=(224, 224), shuffle=True, 
                                                       subset='validation', class_mode='categorical')


Found 21783 images belonging to 3 classes.


In [96]:
test_data = train_data_generator.flow_from_directory('data/train/', target_size=(224, 224), shuffle=True, 
                                                       class_mode='categorical')


Found 72614 images belonging to 3 classes.


In [97]:
history = model.fit(train_data, epochs=15, steps_per_epoch=32, 
                                    validation_data=validation_data, validation_steps=5,
                                    callbacks=[early_stop, checkpoint])

Epoch 1/15
32/32 [==============================] - ETA: 0s - loss: 0.6347 - accuracy: 0.7871
Epoch 1: val_loss improved from inf to 0.44789, saving model to inceptionv3_best.h5
32/32 [==============================] - 24s 660ms/step - loss: 0.6347 - accuracy: 0.7871 - val_loss: 0.4479 - val_accuracy: 0.8875
Epoch 2/15
32/32 [==============================] - ETA: 0s - loss: 0.3169 - accuracy: 0.9170
Epoch 2: val_loss did not improve from 0.44789
32/32 [==============================] - 29s 905ms/step - loss: 0.3169 - accuracy: 0.9170 - val_loss: 0.5962 - val_accuracy: 0.8813
Epoch 3/15
32/32 [==============================] - ETA: 0s - loss: 0.3425 - accuracy: 0.9082
Epoch 3: val_loss improved from 0.44789 to 0.22383, saving model to inceptionv3_best.h5
32/32 [==============================] - 29s 918ms/step - loss: 0.3425 - accuracy: 0.9082 - val_loss: 0.2238 - val_accuracy: 0.9375
Epoch 4/15
32/32 [==============================] - ETA: 0s - loss: 0.3172 - accuracy: 0.8984
Epoch 4: 

In [72]:
score = model.evaluate(test_data)
score

 357/2270 [===>..........................] - ETA: 28:02 - loss: 0.3319 - accuracy: 0.9038

KeyboardInterrupt: 

In [98]:
import cv2
import numpy as np
class_names = ['cat', 'dog', 'person']  
def predict(image_file):
    image = cv2.imread(image_file)
    image = cv2.resize(image, (224,224))
    image = image / 255.0  # Normalize pixel values
    image = np.expand_dims(image, axis=0)  # Add a batch dimension
    predictions = model.predict(image)
    top_classes = np.argsort(predictions[0])[::-1][:5]  # Get the top 5 predicted classes

    for class_idx in top_classes:
        class_name = class_names[class_idx]
        probability = predictions[0][class_idx]
        print(f"Class: {class_name}, Probability: {probability:.4f}")


In [99]:
predict("person.jpg")

1/1 [==============================] - 1s 1s/step
Class: person, Probability: 0.9999
Class: dog, Probability: 0.0001
Class: cat, Probability: 0.0000


In [100]:
predict("dog.jpg")

1/1 [==============================] - 0s 38ms/step
Class: dog, Probability: 0.8058
Class: person, Probability: 0.1427
Class: cat, Probability: 0.0514


In [101]:
predict("cat1.jpg")

1/1 [==============================] - 0s 38ms/step
Class: cat, Probability: 0.9451
Class: person, Probability: 0.0548
Class: dog, Probability: 0.0001


In [102]:
predict("cat2.jpg")

1/1 [==============================] - 0s 40ms/step
Class: cat, Probability: 0.9821
Class: person, Probability: 0.0147
Class: dog, Probability: 0.0031


In [103]:
tflite_converter = tf.lite.TFLiteConverter.from_keras_model(model)

#convert
tflite_model = tflite_converter.convert()
open("tf_lite_model.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Tea_W\AppData\Local\Temp\tmpx3scht95\assets


INFO:tensorflow:Assets written to: C:\Users\Tea_W\AppData\Local\Temp\tmpx3scht95\assets


11488568